In [ ]:
import pandas as pd
import streamlit as st
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from streamlit_folium import st_folium

df = pd.read_csv("../Data/ModelingDataset.csv", low_memory=False)
df2 = pd.read_csv("../Data/Data0/LFB incident et mobilisation data.csv", low_memory=False)


In [4]:
df2.head()

,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,DayOfWeek,Month,IncidentGroup,DetailedIncidentGroup,PropertyCategory,...,TravelTimeSeconds,TotalResponseTime,DeployedFromStation_Code,DeployedFromStation_Name,FromHomeStation,TopDelay,IncGeo_Rounded,Lat_station,Long_station,distance
0,8302309062022,2022-06-09,2022,22:01:18,22,Thursday,June,Fire,Primary Fire,Road Vehicle,...,4.0,5.0,G56,Heathrow,0,0,0,51.480169,-0.458933,3770.970
1,17754712112023,2023-11-12,2023,18:27:59,18,Sunday,November,False Alarm,False alarm - Good intent,Road Vehicle,...,591.0,648.0,G56,Heathrow,1,1,0,51.480169,-0.458933,3781.096
2,17215519102022,2022-10-19,2022,10:52:46,10,Wednesday,October,Fire,Secondary Fire,Outdoor,...,453.0,478.0,G56,Heathrow,1,1,0,51.480169,-0.458933,3785.636
3,11737431082019,2019-08-31,2019,19:58:21,19,Saturday,August,Fire,Primary Fire,Road Vehicle,...,341.0,446.0,G56,Heathrow,1,1,0,51.480169,-0.458933,3802.989
4,4273626032022,2022-03-26,2022,16:31:06,16,Saturday,March,Fire,Secondary Fire,Outdoor,...,422.0,464.0,G56,Heathrow,1,1,0,51.480169,-0.458933,3757.621


In [13]:
# Fusion des deux DataFrames sur 'IncidentNumber'
df_merged = pd.merge(df, df2, on='IncidentNumber', how='inner')

# Filtrer pour l'année 2024
df_2024 = df_merged[df_merged['CalYear_x'] == 2024]


In [14]:
# Select the desired columns and create df_carte
var_interet = ['TotalResponseTime','ResponseTimeBinary', 'inner','Latitude', 'Longitude',
               'IncidentStationGround','DeployedFromStation_Name',
                'DayOfWeek','HourOfCall','DetailedIncidentGroup']
df_2024 = df_2024[var_interet]

In [24]:
df_2024['IncidentStationGround'] = df_2024['IncidentStationGround'].str.lower()
df_2024['DeployedFromStation_Name'] = df_2024['DeployedFromStation_Name'].str.lower()

In [35]:
station = pd.read_csv('..\Data\FireStationInfo_2.csv',sep = ";", low_memory=False)
# Sélection des colonnes souhaitées
station = station[["Station name", "Latitude", "Longitude"]]

# Conversion en majuscules de la colonne "Station name"
station["Station name"] = station["Station name"].str.lower()


In [36]:
station = station[["Station name", "Latitude", "Longitude"]]
station["Station name"] = station["Station name"].str.lower()

In [ ]:
def generer_carte(choix_caserne,df_2024, station, response_time_binary_filter=None):

    # Choix de la partie du dataframe a considérer
    if choix_caserne != "all":
        df = df_2024[df_2024["IncidentStationGround"] == choix_caserne]
    else:
        df = df_2024

    # Nommage de la carte
    if response_time_binary_filter == 1:
        df = df[df["ResponseTimeBinary"] == 1]
        nom_fichier = choix_caserne + "1.html"
    else :
        nom_fichier = choix_caserne + ".html"

    # Récupération des infos sur la caserne
    station_caserne = station[station["Station name"] == choix_caserne]

    # Calcule des coordonnées de centrage de la carte
    if not df.empty:
        mean_lat = df["Latitude"].mean()
        mean_lon = df["Longitude"].mean()
    elif not station_caserne.empty:
        mean_lat = station_caserne["Latitude"].values[0]
        mean_lon = station_caserne["Longitude"].values[0]
    else:
        st.warning("Aucune donnée disponible pour cette caserne.")
        return

    m = folium.Map(location=[mean_lat, mean_lon], zoom_start=12)

    marker_cluster = MarkerCluster().add_to(m)

    for _, row in df.iterrows():
        color = "red" if row["ResponseTimeBinary"] == 1 else "green"
        # Création des attribus de l'étiquette
        if row['IncidentStationGround'] == row['DeployedFromStation_Name']:
            popup_content = f"""
                <div style="font-size: 16px; width: 300px; height: 200px; overflow: auto;">
                    Temps de réponse : {row['TotalResponseTime']}<br>
                    Responsable : {row['IncidentStationGround']}<br>
                    Type : {row['DetailedIncidentGroup']}<br>
                    Jour : {row['DayOfWeek']}<br>
                    Heure : {row['HourOfCall']}<br>
                </div>
            """
        else:
            popup_content = f"""
                <div style="font-size: 16px; width: 300px; height: 200px; overflow: auto;">
                    Temps de réponse : {row['TotalResponseTime']}<br>
                    Responsable : {row['IncidentStationGround']}<br>
                    Deployée : {row['DeployedFromStation_Name']}<br>
                    Type : {row['DetailedIncidentGroup']}<br>
                    Jour : {row['DayOfWeek']}<br>
                    Heure : {row['HourOfCall']}<br>
                </div>
            """

        folium.Marker(
            location=[row["Latitude"], row["Longitude"]],
            popup=popup_content,
            icon=folium.Icon(color=color),
        ).add_to(marker_cluster)

        # Tracer une ligne entre la station déployée et la station responsable
        if row['IncidentStationGround'] != row['DeployedFromStation_Name']:
            deployed_station_name = row['DeployedFromStation_Name']
            deployed_station = station[station["Station name"] == deployed_station_name]

            if not deployed_station.empty:
                    deployed_lat = deployed_station["Latitude"].values[0]
                    deployed_lon = deployed_station["Longitude"].values[0]

                    folium.PolyLine(
                        locations=[(row["Latitude"], row["Longitude"]), (deployed_lat, deployed_lon)],
                        color="black",
                        linestyle='dotted',
                        weight=2.5,
                        opacity=0.8,
                    ).add_to(m)

            if not deployed_station.empty:
            # Récupérer le nom de la caserne déployée
                deployed_station_name = row['DeployedFromStation_Name']

                folium.Marker(
                    location=[deployed_station["Latitude"].values[0], deployed_station["Longitude"].values[0]],
                    popup=f"Caserne déployée : {deployed_station_name}",  # Utiliser deployed_station_name
                    tooltip="Caserne déployée",
                    icon=folium.Icon(color="black", icon="home"),
                ).add_to(m)

    if not station_caserne.empty:
        folium.Marker(
            location=[station_caserne["Latitude"].values[0], station_caserne["Longitude"].values[0]],
            popup=f"Caserne : {choix_caserne}",
            tooltip="Caserne",
            icon=folium.Icon(color="black", icon="home"),
        ).add_to(m)

    st_folium(m)

    m.save(nom_fichier)

In [48]:
caserne = "ruislip"
generer_carte(caserne,df_2024, station)


2025-02-21 08:21:52.470 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-21 08:21:52.472 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-21 08:21:52.474 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-21 08:21:52.476 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
for caserne in df_2024["IncGeo_BoroughName"].unique():
    generer_carte(caserne)
    generer_carte(caserne,1)